# Ćwiczenie: Budowa core DataBota

**Czas trwania:** 90 minut

**Cel ćwiczenia:** W tym ćwiczeniu stworzymy szkielet DataBota, który będzie w stanie przetwarzać zapytania użytkownika, pobierać odpowiednie dane i generować odpowiedzi przy użyciu modelu LLM.

## 1. Wprowadzenie do przetwarzania zapytań użytkownika

Nasza architektura DataBota będzie oparta na następującym pipeline przetwarzania zapytań:

1. Użytkownik wprowadza zapytanie w języku naturalnym
2. Zapytanie jest konwertowane na wektor embeddings
3. System wykonuje wyszukiwanie semantyczne w bazie danych wektorowej
4. Znalezione informacje są przekazywane jako kontekst do modelu LLM
5. Model LLM generuje odpowiedź na podstawie zapytania i kontekstu

Ta architektura, znana jako Retrieval Augmented Generation (RAG), pozwala na wykorzystanie zewnętrznych źródeł danych w procesie generowania odpowiedzi, co znacznie zwiększa dokładność i aktualność informacji.

In [ ]:
# Instalacja wymaganych bibliotek
%pip install openai langchain tiktoken pymupdf 

In [ ]:


# Import bibliotek
import openai
import numpy as np
import pandas as pd
from langchain import LLMChain
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import AzureOpenAI
from langchain.prompts import PromptTemplate
import os
import json
import time

# Sprawdzenie, czy wszystkie potrzebne biblioteki zostały poprawnie zaimportowane
print("Środowisko zostało zainicjalizowane pomyślnie")

## 2. Konfiguracja dostępu do Azure OpenAI

Aby korzystać z modeli OpenAI, musimy skonfigurować dostęp do Azure OpenAI Service.

In [ ]:
import os
from openai import AzureOpenAI

endpoint = "https://aissqldayedbworkshop001.openai.azure.com/"
model_name = "o3-mini"
deployment = "o3-mini"

subscription_key="4mE2kj9PLeZ0NjqMiFzgxtStKtJIDRnZ4dzNIsipygEDdbYHmlCXJQQJ99BEAC5RqLJXJ3w3AAAAACOG85N1"
api_version = "2024-12-01-preview"

client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=endpoint,
    api_key=subscription_key,
)

response = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant.",
        },
        {
            "role": "user",
            "content": "I am going to Paris, what should I see?",
        }
    ],
    max_completion_tokens=100000,
    model=deployment
)

print(response.choices[0].message.content)



In [ ]:
# Konfiguracja dostępu do Azure OpenAI
# W środowisku produkcyjnym te dane powinny być przechowywane w Azure Key Vault lub Databricks Secrets

# Dla Azure OpenAI Service
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-05-15"  # Dostosuj do aktualnej wersji API
os.environ["OPENAI_API_BASE"] = "https://your-azure-openai-resource.openai.azure.com/"  # Zastąp swoim URL
os.environ["OPENAI_API_KEY"] = "your-azure-openai-key"  # Zastąp swoim kluczem API

# Weryfikacja konfiguracji
def test_openai_connection():
    try:
        # Inicjalizacja klienta OpenAI
        openai.api_type = os.environ["OPENAI_API_TYPE"]
        openai.api_base = os.environ["OPENAI_API_BASE"]
        openai.api_version = os.environ["OPENAI_API_VERSION"]
        openai.api_key = os.environ["OPENAI_API_KEY"]
        
        # Próba wykonania prostego zapytania
        response = openai.Completion.create(
            engine="text-davinci-003",  # Dostosuj do dostępnego modelu
            prompt="Hello, world!",
            max_tokens=5
        )
        print("Połączenie z Azure OpenAI działa poprawnie")
        return True
    except Exception as e:
        print(f"Błąd połączenia z Azure OpenAI: {str(e)}")
        return False

# Uruchom test połączenia
connection_ok = test_openai_connection()

### 🔧 Zadanie dla uczestnika: Generowanie embeddings
Użyj modelu embeddingowego z Azure OpenAI do przetworzenia wczytywanych dokumentów na wektory. 

**Przykład:**
```python
response = openai.Embedding.create(
    input=documents,
    engine="text-embedding-ada-002"
)
embeddings = [e["embedding"] for e in response["data"]]
```

In [ ]:
documents = [
    "Databricks to platforma analityczna oparta na Apache Spark. Oferuje zintegrowane środowisko do analizy danych, uczenia maszynowego i wizualizacji.",
    "Azure OpenAI Service udostępnia modele GPT-4, GPT-3.5 Turbo i inne w chmurze Microsoft Azure. Zapewnia zaawansowane funkcje przetwarzania języka naturalnego.",
    "Retrieval Augmented Generation (RAG) to technika łącząca wyszukiwanie informacji z generacją tekstu. Pozwala na wykorzystanie zewnętrznych źródeł wiedzy w modelach generatywnych.",
    "Microsoft Fabric to zintegrowana platforma analityczna, która łączy różne usługi analityczne w jednym miejscu. Obejmuje Data Engineering, Data Factory, Synapse Data Science i inne.",
    "Wektorowe bazy danych, takie jak FAISS (Facebook AI Similarity Search), umożliwiają efektywne przechowywanie i wyszukiwanie wektorów embeddingowych reprezentujących teksty lub obrazy."
]

In [ ]:
endpoint = "https://aissqldayedbworkshop002.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-05-15"
api_key = "BCg7xEV4SUnG4QKIKi6rEYEnlZbzKzbpynNDh4XM1QX1BKDzJ6pfJQQJ99BEAC5RqLJXJ3w3AAAAACOGlwwk"

client = openai.AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=api_key,
    api_version="2023-09-01-preview"
)

In [ ]:
import openai

response = client.embeddings.create(
    input=documents,
    model="text-embedding-ada-002"
)
#embeddings = [e["embedding"] for e in response["data"]]

### 🔧 Zadanie dla uczestnika: Wyszukiwanie wektorowe
Zaimplementuj funkcję, która przyjmie zapytanie użytkownika, wygeneruje jego embedding i znajdzie najbardziej podobne dokumenty przy użyciu metryki kosinusowej.

**Przykład:**
```python
# Funkcja porównująca wektory
from sklearn.metrics.pairwise import cosine_similarity

query = "Jak działa MLflow?"
query_emb = embedding_function(query)
similarities = cosine_similarity([query_emb], embeddings)
```

### 🔧 Zadanie dla uczestnika: Generowanie odpowiedzi z LLM
Wygeneruj prompt łączący kontekst znalezionych dokumentów z zapytaniem użytkownika i prześlij go do Azure OpenAI.

**Przykład:**
```python
prompt = f"Odpowiedz na pytanie w oparciu o poniższy kontekst:\n{top_docs}\n\nPytanie: {query}"
response = openai.ChatCompletion.create(
    engine="gpt-4",
    messages=[{"role": "user", "content": prompt}]
)
print(response["choices"][0]["message"]["content"])
```

In [ ]:
# Przykładowe dane testowe
sample_documents = [
    "Databricks to platforma analityczna oparta na Apache Spark. Oferuje zintegrowane środowisko do analizy danych, uczenia maszynowego i wizualizacji.",
    "Azure OpenAI Service udostępnia modele GPT-4, GPT-3.5 Turbo i inne w chmurze Microsoft Azure. Zapewnia zaawansowane funkcje przetwarzania języka naturalnego.",
    "Retrieval Augmented Generation (RAG) to technika łącząca wyszukiwanie informacji z generacją tekstu. Pozwala na wykorzystanie zewnętrznych źródeł wiedzy w modelach generatywnych.",
    "Microsoft Fabric to zintegrowana platforma analityczna, która łączy różne usługi analityczne w jednym miejscu. Obejmuje Data Engineering, Data Factory, Synapse Data Science i inne.",
    "Wektorowe bazy danych, takie jak FAISS (Facebook AI Similarity Search), umożliwiają efektywne przechowywanie i wyszukiwanie wektorów embeddingowych reprezentujących teksty lub obrazy."
]

# Przykładowe dane tabelaryczne w formacie DataFrame
sample_structured_data = pd.DataFrame({
    'Product': ['Laptop', 'Smartphone', 'Tablet', 'Monitor', 'Keyboard'],
    'Price': [5000, 3000, 2000, 1500, 300],
    'Inventory': [120, 200, 80, 50, 300],
    'Category': ['Electronics', 'Electronics', 'Electronics', 'Accessories', 'Accessories'],
    'Description': [
        'Wydajny laptop do zastosowań biznesowych z procesorem Intel i7',
        'Smartfon z ekranem dotykowym i potrójnym aparatem',
        'Lekki tablet z długim czasem pracy na baterii',
        'Monitor 4K z wysoką częstotliwością odświeżania',
        'Ergonomiczna klawiatura mechaniczna'
    ]
})

# Wyświetlenie przykładowych danych
print("Przykładowe dokumenty:")
for i, doc in enumerate(sample_documents):
    print(f"[{i}] {doc[:100]}...")

print("\nPrzykładowe dane strukturalne:")
display(sample_structured_data)

### 🔧 Zadanie dla uczestnika: Mechanizm pamięci (memory)
Utwórz prostą strukturę przechowującą historię rozmowy (np. listę słowników). Dołączaj ją do promptu, aby model miał kontekst wcześniejszych pytań i odpowiedzi.

**Przykład:**
```python
memory = [
    {"role": "user", "content": "Jak działa Databricks?"},
    {"role": "assistant", "content": "Databricks to platforma analityczna oparta na Apache Spark..."}
]
```

### 🔧 Zadanie dla uczestnika: Integracja całego pipeline'u
Połącz wszystkie komponenty w jedną funkcję lub klasę DataBota. Przetestuj ją na kilku różnych zapytaniach i obserwuj odpowiedzi modelu.

**Przykład:**
```python
def databot_respond(query, memory):
    # implementacja pipeline: embedding → wyszukiwanie → prompt → odpowiedź
    pass
```

## 6. Implementacja wyszukiwania wektorowego

Teraz zaimplementujemy funkcję wyszukiwania semantycznego, która będzie wykorzystywać bazę wektorową do znalezienia najbardziej odpowiednich fragmentów dokumentów dla danego zapytania.

In [ ]:
# Implementacja funkcji wyszukiwania wektorowego
def semantic_search(query, vectordb, top_k=3):
    # Wykonaj wyszukiwanie wektorowe
    results = vectordb.similarity_search_with_score(query, k=top_k)
    
    # Formatuj wyniki
    formatted_results = []
    for doc, score in results:
        formatted_results.append({
            "text": doc.page_content,
            "source": doc.metadata.get("source", "unknown"),
            "similarity_score": float(score)
        })
    
    return formatted_results

# Testowanie funkcji wyszukiwania
test_query = "Czym jest RAG i jak działa?"
try:
    search_results = semantic_search(test_query, vectordb)
    print(f"Wyniki wyszukiwania dla zapytania: '{test_query}'\n")
    for i, result in enumerate(search_results):
        print(f"Wynik {i+1} (score: {result['similarity_score']:.4f})")
        print(f"Źródło: {result['source']}")
        print(f"Tekst: {result['text']}\n")
except Exception as e:
    print(f"Błąd podczas wyszukiwania: {str(e)}")

## 7. Integracja z Azure OpenAI do generowania odpowiedzi

Teraz zintegrujemy nasze wyszukiwanie semantyczne z modelem LLM, aby generować odpowiedzi na podstawie znalezionych fragmentów dokumentów.

In [ ]:
# Implementacja funkcji generowania odpowiedzi przy użyciu Azure OpenAI
def generate_response(query, search_results):
    # Inicjalizacja modelu LLM
    llm = AzureOpenAI(
        deployment_name="gpt-4",  # Dostosuj do dostępnego modelu
        model_name="gpt-4",
        openai_api_key=os.environ["OPENAI_API_KEY"],
        openai_api_base=os.environ["OPENAI_API_BASE"],
        openai_api_version=os.environ["OPENAI_API_VERSION"]
    )
    
    # Przygotuj kontekst z wyników wyszukiwania
    context = "\n\n".join([f"Źródło: {r['source']}\nTekst: {r['text']}" for r in search_results])
    
    # Przygotuj szablon promptu
    template = """
    Odpowiedz na poniższe pytanie, korzystając tylko z informacji zawartych w dostarczonym kontekście.
    Jeśli nie możesz znaleźć odpowiedzi w kontekście, powiedz "Nie znalazłem odpowiedzi w dostępnych danych".
    
    Kontekst:
    {context}
    
    Pytanie: {query}
    
    Odpowiedź:
    """
    
    prompt = PromptTemplate(template=template, input_variables=["context", "query"])
    
    # Utwórz łańcuch LLM
    chain = LLMChain(prompt=prompt, llm=llm)
    
    # Generuj odpowiedź
    response = chain.run(context=context, query=query)
    
    return response.strip()

# Testowanie funkcji generowania odpowiedzi
try:
    response = generate_response(test_query, search_results)
    print(f"Odpowiedź na pytanie: '{test_query}'\n")
    print(response)
except Exception as e:
    print(f"Błąd podczas generowania odpowiedzi: {str(e)}")

## 8. Implementacja prostego mechanizmu śledzenia rozmowy (memory)

Aby DataBot mógł prowadzić spójną konwersację, musimy zaimplementować mechanizm śledzenia rozmowy.

In [ ]:
# Implementacja prostego mechanizmu śledzenia rozmowy
class ConversationMemory:
    def __init__(self, max_history=5):
        self.conversation_history = []
        self.max_history = max_history
    
    def add_interaction(self, query, response):
        self.conversation_history.append({"query": query, "response": response, "timestamp": time.time()})
        # Ogranicz historię do max_history ostatnich interakcji
        if len(self.conversation_history) > self.max_history:
            self.conversation_history = self.conversation_history[-self.max_history:]
    
    def get_history(self):
        return self.conversation_history
    
    def get_formatted_history(self):
        formatted = ""
        for interaction in self.conversation_history:
            formatted += f"User: {interaction['query']}\n"
            formatted += f"Assistant: {interaction['response']}\n\n"
        return formatted
    
    def clear_history(self):
        self.conversation_history = []

# Inicjalizacja pamięci konwersacji
memory = ConversationMemory()

# Dodanie pierwszej interakcji do pamięci
memory.add_interaction(test_query, response)

# Wyświetlenie historii konwersacji
print("Historia konwersacji:")
print(memory.get_formatted_history())

## 9. Integracja wszystkich komponentów w kompletnym pipeline'ie

Teraz połączymy wszystkie komponenty w jeden kompletny pipeline DataBota.

In [ ]:
# Integracja wszystkich komponentów w klasie DataBot
class DataBot:
    def __init__(self, documents=None, structured_data=None):
        # Inicjalizacja pamięci konwersacji
        self.memory = ConversationMemory()
        
        # Inicjalizacja bazy wektorowej dla dokumentów
        if documents:
            self.vectordb = generate_embeddings(documents)
        else:
            self.vectordb = None
        
        # Przechowywanie danych strukturalnych
        self.structured_data = structured_data
        
        print("DataBot został zainicjalizowany")
    
    def process_query(self, query, use_memory=True, top_k=3):
        # Rozszerzenie zapytania o kontekst z historii konwersacji, jeśli włączone
        if use_memory and self.memory.get_history():
            context_query = f"Biorąc pod uwagę poprzednią konwersację:\n{self.memory.get_formatted_history()}\nOdpowiedz na pytanie: {query}"
        else:
            context_query = query
        
        # Wyszukiwanie semantyczne w dokumentach
        if self.vectordb:
            search_results = semantic_search(query, self.vectordb, top_k=top_k)
        else:
            search_results = []
        
        # Dodanie danych strukturalnych, jeśli pytanie może ich dotyczyć
        # To jest uproszczona implementacja - w rzeczywistości potrzebny byłby bardziej zaawansowany mechanizm
        if self.structured_data is not None and any(keyword in query.lower() for keyword in ["produkt", "cena", "inventory", "kategoria"]):
            # Konwersja DataFrame na tekstową reprezentację
            structured_text = self.structured_data.to_string()
            search_results.append({
                "text": f"Dane produktowe:\n{structured_text}",
                "source": "structured_data",
                "similarity_score": 1.0  # Przypisujemy wysoki wynik, ponieważ jawnie włączamy te dane
            })
        
        # Generowanie odpowiedzi
        response = generate_response(context_query, search_results)
        
        # Dodanie interakcji do pamięci
        self.memory.add_interaction(query, response)
        
        return {
            "query": query,
            "search_results": search_results,
            "response": response
        }
    
    def clear_memory(self):
        self.memory.clear_history()
        print("Historia konwersacji została wyczyszczona")

# Inicjalizacja DataBota z naszymi przykładowymi danymi
databot = DataBot(documents=sample_documents, structured_data=sample_structured_data)

# Testowanie DataBota
test_queries = [
    "Czym jest RAG i jak to pomaga w AI?",
    "Które produkty są dostępne w kategorii Electronics?",
    "Możesz powiedzieć więcej o Microsoft Fabric?"
]

for i, query in enumerate(test_queries):
    print(f"\n--- Zapytanie {i+1}: {query} ---")
    result = databot.process_query(query)
    print("\nOdpowiedź:")
    print(result["response"])

print("\n--- Historia konwersacji ---")
print(databot.memory.get_formatted_history())

## 12. Zadania do wykonania

1. Zmodyfikuj promptu dla LLM, aby poprawić jakość generowanych odpowiedzi
2. Dodaj obsługę odpowiedzi na pytania, na które nie ma informacji w dostępnych danych
3. Zaimplementuj mechanizm filtrowania dokumentów na podstawie ich wyników podobieństwa (np. ustaw próg minimalnego podobieństwa)
4. Dodaj obsługę zapytań follow-up, które odnoszą się do wcześniejszych odpowiedzi

## Zadania dodatkowe

Jeśli masz więcej czasu, możesz spróbować:

1. Zaimplementowanie mechanizmu oceny jakości odpowiedzi (evaluation)
2. Dodanie obsługi różnych typów zapytań (np. zapytania o dane, zapytania o wiedzę ogólną, zapytania o akcje)
3. Implementacja bardziej zaawansowanego promptu dla LLM z wykorzystaniem Chain of Thought
4. Zaimplementowanie mechanizmu wyjaśnialności odpowiedzi (np. podanie źródeł informacji)